In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import json
import os
import yaml
from dask.distributed import Client

In [5]:
!ls -l ~/.dlio_profiler/

total 8
-rwx------ 1 kogiou1 kogiou1 659 Nov  2 10:42 configuration.sh
-rw------- 1 kogiou1 kogiou1  79 Nov  2 10:43 configuration.yaml


In [6]:
with open('/g/g92/kogiou1/.dlio_profiler/configuration.yaml', 'r') as file:
    dlp_yaml = yaml.safe_load(file)

In [7]:
# to make this notebook in dev mode.
import sys
sys.path.insert(0,dlp_yaml["app"])
sys.path

['/usr/workspace/iopp/software/dlio-profiler',
 '/usr/WS2/iopp/software/dlio-profiler/examples/dlp_analyzer',
 '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python39.zip',
 '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python3.9',
 '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python3.9/lib-dynload',
 '',
 '/usr/workspace/iopp/software/dlio-profiler/venv/lib/python3.9/site-packages']

In [8]:
import dlp_analyzer
print(dlp_analyzer.__file__)
from dlp_analyzer.main import DLPAnalyzer,get_dlp_configuration,update_dlp_configuration,setup_logging,setup_dask_cluster, reset_dask_cluster, get_dlp_configuration

/usr/workspace/iopp/software/dlio-profiler/dlp_analyzer/__init__.py


In [9]:
dask_run_dir = os.path.join(dlp_yaml["app"], "dlp_analyzer", "dask", "run_dir")
dask_run_dir

'/usr/workspace/iopp/software/dlio-profiler/dlp_analyzer/dask/run_dir'

In [10]:
with open (os.path.join(dask_run_dir, "scheduler.json"), "r") as f:
    dask_scheduler = json.load(f)
dask_scheduler

FileNotFoundError: [Errno 2] No such file or directory: '/usr/workspace/iopp/software/dlio-profiler/dlp_analyzer/dask/run_dir/scheduler.json'

In [10]:
app_name = "dlio_cosmoflow" # resnet cosmoflow unet3d mummi deepspeed dlio_resnet dlio_cosmoflow

In [11]:
def get_conditions_cosmoflow(json_object):
    app_io_cond = "TFReader.parse_image" in json_object["name"] # Cosmoflow
    compute_cond = "compute" in json_object["name"] # Cosmoflow
    io_cond = "POSIX" == json_object["cat"] # Cosmoflow
    return app_io_cond, compute_cond, io_cond

def get_conditions_resnet(json_object):
    app_io_cond = "IO" == json_object["cat"] # Resnet50
    compute_cond = "cpu" in json_object["name"] or "compute" in json_object["cat"] # Resnet50
    io_cond = "POSIX" == json_object["cat"] # Cosmoflow
    return app_io_cond, compute_cond, io_cond

def get_conditions_unet3d(json_object):
    app_io_cond = "NPZReader.read_index" in json_object["name"] # Unet3d
    compute_cond = "compute" in json_object["name"] # Unet3d
    io_cond = "POSIX" == json_object["cat"] # Cosmoflow
    return app_io_cond, compute_cond, io_cond

def get_conditions_dlio_resnet(json_object):
    app_io_cond = "read_index" in json_object["name"] # Unet3d
    compute_cond = "compute" in json_object["name"] # Unet3d
    io_cond = "POSIX" == json_object["cat"] # Cosmoflow
    return app_io_cond, compute_cond, io_cond


In [12]:
condition_fn = None #
if app_name == "cosmoflow":
    filename = "/usr/WS2/iopp/kogiou1/dlio_benchmark/hydra_log/cosmoflow/2023-10-31-10-27-24/.trace*.pfw.gz"
    condition_fn = get_conditions_cosmoflow
elif app_name == "dlio_cosmoflow":
    filename = "/usr/workspace/iopp/kogiou1/dlio_benchmark/hydra_log/cosmoflow/2023-11-01-18-47-58/.trace*.pfw.gz"
    condition_fn = get_conditions_cosmoflow
elif app_name == "resnet":
    filename = "/usr/workspace/iopp/dlio_paper_results/resnet_50_1node_4ppn/*.pfw.gz"
    condition_fn = get_conditions_resnet
elif app_name == "unet3d":
    filename = "/usr/workspace/iopp/kogiou1/dlio_benchmark_logs/unet3d/node_32_ppn_4/.trace*.pfw.gz"
    condition_fn = get_conditions_unet3d
elif app_name == "mummi":
    filename = "/usr/workspace/iopp/dlp_traces/mummi-32-node/trace*.pfw.gz"
elif app_name == "deepspeed":
    filename = "/usr/workspace/iopp/dlp_traces/deepspeed_8_4ppn/*.pfw.gz"
elif app_name == "dlio_resnet":
    filename = "/usr/workspace/iopp/kogiou1/dlio_benchmark/hydra_log/resnet50/2023-11-01-10-52-58/.trace*.pfw.gz"
    condition_fn = get_conditions_dlio_resnet
else:
    raise Exception("Unknown App name")

In [13]:
conf = update_dlp_configuration(dask_scheduler=dask_scheduler["address"], verbose=True, 
                                log_file="./dlp.log", rebuild_index=False, time_approximate=True, 
                                host_pattern=r'lassen(\d+)', time_granularity=3600e6, skip_hostname=True, conditions=condition_fn)


NameError: name 'dask_scheduler' is not defined

In [ ]:
conf = get_dlp_configuration()
conf.time_approximate

In [14]:
setup_logging()

In [15]:
setup_dask_cluster()

/usr/workspace/iopp/software/dlio-profiler/venv/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33013 instead
  warnings.warn(


In [16]:
reset_dask_cluster()

2023-11-07 18:24:20,963 - distributed.nanny - WARNING - Restarting worker
2023-11-07 18:24:20,984 - distributed.nanny - WARNING - Restarting worker
2023-11-07 18:24:21,005 - distributed.nanny - WARNING - Restarting worker
2023-11-07 18:24:21,017 - distributed.nanny - WARNING - Restarting worker


In [ ]:
analyzer = DLPAnalyzer(filename)

In [ ]:
analyzer.events.head()

In [ ]:
#analyzer.events[["io_time","compute_time","app_io_time"]].max().compute() / 1e6

In [ ]:
# hydra_log/cosmoflow/2023-11-01-18-47-58/
items = analyzer.summary()
items

In [ ]:
analyzer.events.groupby("name")["size"].sum().compute() / 1024**3

In [ ]:
analyzer.events.groupby(["trange","pid","tid", "name"])["io_time"].sum().groupby(["trange", "name"]).max().groupby([ "name"]).sum().compute() / 1e6

In [ ]:
analyzer.events.groupby(["trange","pid","tid", "name"])["io_time"].count().groupby(["trange", "name"]).sum().groupby([ "name"]).sum().compute()

In [ ]:
timeline = analyzer.plots._create_timeline( analyzer.events).compute()

In [ ]:
timeline

In [ ]:
fig, ax1, ax2 = analyzer.plots.time_bw_timeline(
    bw_unit='gb',
    figsize=(8, 3),
    line1_label='POSIX I/O Time',
    line2_label='POSIX I/O Bandwidth',
    time_col='io_time',
    x_num_ticks=8,
    y_num_ticks=5,
)

In [ ]:
fig.savefig(f'{app_name}_posix_io_time.pdf', format='pdf', bbox_inches='tight')

In [ ]:
fig, ax1, ax2 = analyzer.plots.time_bw_timeline(
    bw_unit='gb',
    figsize=(8, 3),
    line1_label='App. I/O Time',
    line2_label='App. I/O Bandwidth',
    time_col='app_io_time',
    x_num_ticks=8,
    y_num_ticks=5,
)

In [ ]:
fig.savefig(f'{app_name}_app_io_time.pdf', format='pdf', bbox_inches='tight')

In [ ]:
fig, ax = analyzer.plots.xfer_size_timeline(
    figsize=(8, 3),
    unit='kb',
    x_num_ticks=8,
)

In [ ]:
fig.savefig(f'{app_name}_xfer_size.pdf', format='pdf', bbox_inches='tight')

In [ ]:
analyzer.events.query("cat == 'POSIX'").groupby("name")["size"].mean().compute() / (1024**2)

In [ ]:
analyzer.events.query("cat == 'POSIX'").groupby(["trange","pid","tid","name"])["io_time"].sum().groupby(["trange","name"]).max().groupby("name").sum().compute() / 1e6

In [ ]:
val = analyzer.events.groupby(["trange"]).agg({"io_time":[min,max],"compute_time":[min,max],"app_io_time":[min,max]}).sum().compute()
val

In [ ]:
val = analyzer.events["app_io_time"].max().compute()
val

In [ ]:
import dask
val1, val2 = dask.compute(analyzer.events["ts"].min() /1e6 , analyzer.events["ts"].max()/1e6)
val2 - val1

In [ ]:
analyzer.events["hostname"].unique().compute()[:2]

In [ ]:
analyzer.events.query("cat == 'POSIX'").groupby(["trange","name"])["dur"].max().sum().compute() / 1e6

In [ ]:
analyzer.events.query("name == 'open'").groupby("trange")["dur"].max().sum().compute() / 1e6

In [ ]:
analyzer.events.groupby(["cat","name"])["dur"].sum().compute()

In [ ]:
3766287/3.8/1e6, 1022647/3.8/1e6, 1080267/3.8/1e6, 178451/3.8/1e6, (1259124+153089)/3.8/1e6

In [ ]:
analyzer.plots.bottleneck_timeline(figsize=(8, 4))

In [ ]:
analyzer.plots.bw_timeline(figsize=(4, 4), unit='KB')

In [ ]:
analyzer.plots.xfer_size_distribution(figsize=(4, 4))